In [28]:
!pip install ollama gspread pandas


In [38]:
import ollama
from codecarbon import EmissionsTracker

In [39]:
modelfile='''
FROM llama2-uncensored
SYSTEM You are a poet but you like to keep it simple
PARAMETER temperature 5
'''

ollama.create(model='llama2-uncensored:hot', modelfile=modelfile)

{'status': 'success'}

### Generate Poems and track emissions

In [40]:

tracker = EmissionsTracker(save_to_api=True, tracking_mode="process")
model = "llama2-uncensored:hot" # You need to pull the model from the CLI
n_poems = 10
# Start tracking
tracker.start()
poems = []
for i in range(n_poems):
    response = ollama.chat(model=model, messages=[{'role': 'user', 'content': 'Write a poem for me about open source software'}])
    poems.append(response['message']['content'])

emmissions = tracker.stop()

[codecarbon INFO @ 17:00:51] [setup] RAM Tracking...
[codecarbon INFO @ 17:00:51] [setup] GPU Tracking...
[codecarbon INFO @ 17:00:51] No GPU found.
[codecarbon INFO @ 17:00:51] [setup] CPU Tracking...
[codecarbon INFO @ 17:00:52] Tracking Apple CPU and GPU via PowerMetrics
[codecarbon INFO @ 17:00:52] >>> Tracker's metadata:
[codecarbon INFO @ 17:00:52]   Platform system: macOS-12.6-arm64-arm-64bit
[codecarbon INFO @ 17:00:52]   Python version: 3.9.18
[codecarbon INFO @ 17:00:52]   CodeCarbon version: 2.4.3rc1
[codecarbon INFO @ 17:00:52]   Available RAM : 8.000 GB
[codecarbon INFO @ 17:00:52]   CPU count: 8
[codecarbon INFO @ 17:00:52]   CPU model: Apple M1
[codecarbon INFO @ 17:00:52]   GPU count: 1
[codecarbon INFO @ 17:00:52]   GPU model: Apple M1
[codecarbon INFO @ 17:00:53] ApiClient Successfully registered your run on the API.

Run ID: 934a85fd-2b20-4666-aa06-f8b9b002c0c1
Experiment ID: c7ddaa20-db55-411e-aeab-49c2fed4341e

[codecarbon INFO @ 17:01:08] Energy consumed for RAM :

In [41]:
poems

['Source code is written with care,\nAnd every line must be fair.\nThe creator of the code,\nThey offer it out wide and broad.\nOpen source, I can see.\nNo walls to stop the free.\nIn this open source way,\nI will find no dismay.\nLet us all give it a try,\nFor with this source code sky high.',
 'The language that runs our devices, \n\nSoftly speaks without a noise or sign;\n\nOpen Source Software is its name. \n\nIts beauty lies within, hidden behind the flames; \n\nLike fireflies in a forest, twinkling light of code \n\nIs free for all, shared freely for its prowess and pride. \n\nA community working together to create a platform \n\nThat serves humanity with ease without any kind of control;\n\nIts users are the builders, shaping it as they like; \n\nThe source code open for everyone to look into their sight.\n\nIt runs our devices seamlessly and quietly. \n\nThe way its made, so much so we cannot deny, \n\nOpen Source Software, a treasure worth to behold.',
 "In the realm of open s

In [42]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define the scope of the application
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# Provide the path to your service account key
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/luis/Downloads/mozfest-d60a0151e681.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet using its title or URL
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1zGKLYhEkaD4R9l_Rhzs5blmWx608kA6BpzoTIPAO2zs/edit?usp=sharing") 
sheet = spreadsheet.get_worksheet(0)

def build_data_to_insert(poems, model):
    data = {
    'Poem': poems,
    'Description': f"Model: {model}, emissions: {emmissions / n_poems} kg/C02eq",
    "Type": "SCALE",
    "Answer Start": "0,5,BAD,GOOD"
    }
    df =pd.DataFrame(data)

    return df.astype(str).values.tolist()

# Update the sheet with DataFrame data

data_to_append = build_data_to_insert(poems, model)
sheet.append_rows(data_to_append, value_input_option='RAW')


{'spreadsheetId': '1zGKLYhEkaD4R9l_Rhzs5blmWx608kA6BpzoTIPAO2zs',
 'tableRange': 'Sheet1!A1:D42',
 'updates': {'spreadsheetId': '1zGKLYhEkaD4R9l_Rhzs5blmWx608kA6BpzoTIPAO2zs',
  'updatedRange': 'Sheet1!A43:D52',
  'updatedRows': 10,
  'updatedColumns': 4,
  'updatedCells': 40}}